**Authentification Process**

In [2]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


**Enable Table Display**

In [3]:
%load_ext google.colab.data_table

**Load BigQuery Google Cloud Platform Project**

In [5]:
project_id = 'data-engineer-next-1'

from google.cloud import bigquery

client = bigquery.Client(project=project_id)

**Subqueries/Nested Queries + Date and Time**

In [8]:
df = client.query('''
  SELECT
  bike_id,
  start_station_name AS station_name
  FROM
  `bigquery-public-data.austin_bikeshare.bikeshare_trips`
  WHERE
  TIMESTAMP_TRUNC(start_time, DAY) = TIMESTAMP("2015-02-21")
  AND start_station_name IN (
    SELECT
      name
    FROM
      `bigquery-public-data.austin_bikeshare.bikeshare_stations`
    WHERE
      status = 'active'
  ) LIMIT 5;''').to_dataframe()

df.head()

,bike_id,station_name
0,933,Plaza Saltillo
1,397,Plaza Saltillo
2,105,Plaza Saltillo
3,142,Plaza Saltillo
4,957,Plaza Saltillo


**Joins**

In [33]:
df = client.query('''
SELECT
  b.bike_id,
  bk.name AS station_name,  -- Change to station_name
  b.duration_minutes,
  bk.power_type
FROM (
  SELECT
    bike_id,
    start_station_id,
    duration_minutes
  FROM
    `bigquery-public-data.austin_bikeshare.bikeshare_trips`
  WHERE
    duration_minutes > 30
) AS b
LEFT JOIN (
  SELECT
    station_id,
    name,  -- Include station name
    power_type
  FROM
    `bigquery-public-data.austin_bikeshare.bikeshare_stations`
  WHERE
    power_type = 'solar'
) AS bk
ON
  b.start_station_id = bk.station_id
ORDER BY
  b.duration_minutes
LIMIT
  5
''').to_dataframe()

df.head()


,bike_id,station_name,duration_minutes,power_type
0,958,4th/Guadalupe @ Republic Square,31,solar
1,19606,11th & San Jacinto,31,solar
2,19319,Hollow Creek & Barton Hills,31,solar
3,288,17th & Guadalupe,31,solar
4,16974,Rosewood & Angelina,31,solar


**UNION**

In [19]:
df = client.query('''
  (SELECT
  start_station_name AS station_name
  FROM
  `bigquery-public-data.austin_bikeshare.bikeshare_trips`
  LIMIT
  5)
  UNION DISTINCT
  (SELECT
  name
  FROM
  `bigquery-public-data.austin_bikeshare.bikeshare_stations`
  LIMIT
  5)''').to_dataframe()

df.tail()

,station_name
1,8th & Guadalupe
2,Red River & LBJ Library
3,Rainey @ River St
4,State Parking Garage @ Brazos & 18th
5,ACC - West & 12th Street


**AGGREGATE FUNCTION**

In [21]:
df = client.query('''
  SELECT
  start_station_name,
  AVG(duration_minutes) AS average_duration
FROM
  `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY
  start_station_name
ORDER BY
  average_duration DESC
  LIMIT
  5''').to_dataframe()

df.head()

,start_station_name,average_duration
0,East 5th/Shady Ln,197.470588
1,cesar Chavez/Congress,114.000000
2,Stolen,105.000000
3,Lakeshore & Pleasant Valley,71.895495
4,Lakeshore @ Austin Hostel,65.007563


**Window Functions**

In [24]:
df = client.query('''
    SELECT
  start_station_name,
  bike_id,
  duration_minutes,
  RANK() OVER (PARTITION BY start_station_name ORDER BY duration_minutes DESC) AS duration_rank
FROM
  `bigquery-public-data.austin_bikeshare.bikeshare_trips`
ORDER BY
  start_station_name,
  duration_rank
LIMIT
5;''').to_dataframe()

df.head()

,start_station_name,bike_id,duration_minutes,duration_rank
0,10th & Red River,1517,2863,1
1,10th & Red River,397,1720,2
2,10th & Red River,897,1026,3
3,10th & Red River,2126,596,4
4,10th & Red River,2048,507,5


**Common Table Expressions (CTEs)**

In [32]:
df = client.query('''
WITH RankedTrips AS (
  SELECT
    start_station_name,
    bike_id,
    duration_minutes,
    ROW_NUMBER() OVER (PARTITION BY start_station_name ORDER BY duration_minutes DESC) AS duration_rank
  FROM
    `bigquery-public-data.austin_bikeshare.bikeshare_trips`
)

SELECT
  start_station_name,
  bike_id,
  duration_minutes
FROM
  RankedTrips
WHERE
  duration_rank = 1
ORDER BY
  start_station_name
''').to_dataframe()

df.head()


,start_station_name,bike_id,duration_minutes
0,10th & Red River,1517,2863
1,10th/Red River,113G,1728
2,11th & Salina,183,6005
3,11th & San Jacinto,287,3632
4,11th/Congress @ The Texas Capitol,72,9536


**Pivot And Case Statement**

In [26]:
df = client.query('''
SELECT
  start_station_name,
  AVG(CASE WHEN EXTRACT(YEAR FROM start_time) = 2018 THEN duration_minutes END) AS avg_duration_2018,
  AVG(CASE WHEN EXTRACT(YEAR FROM start_time) = 2019 THEN duration_minutes END) AS avg_duration_2019,
  AVG(CASE WHEN EXTRACT(YEAR FROM start_time) = 2020 THEN duration_minutes END) AS avg_duration_2020,
  AVG(CASE WHEN EXTRACT(YEAR FROM start_time) = 2021 THEN duration_minutes END) AS avg_duration_2021,
  AVG(CASE WHEN EXTRACT(YEAR FROM start_time) = 2022 THEN duration_minutes END) AS avg_duration_2022
FROM
  `bigquery-public-data.austin_bikeshare.bikeshare_trips`
GROUP BY
  start_station_name
ORDER BY
  start_station_name
  LIMIT
  5''').to_dataframe()

df.head()

,start_station_name,avg_duration_2018,avg_duration_2019,avg_duration_2020,avg_duration_2021,avg_duration_2022
0,10th & Red River,51.975758,51.322581,NaN,NaN,NaN
1,10th/Red River,NaN,74.393701,40.247059,41.42596,30.290373
2,11th & Salina,54.899128,71.857143,NaN,NaN,NaN
3,11th & San Jacinto,27.062287,53.793991,NaN,NaN,NaN
4,11th/Congress @ The Texas Capitol,NaN,94.329738,57.398570,38.21959,34.198716


**String Manipulation and Recursive Queries**

In [27]:
df = client.query('''
WITH RECURSIVE DurationCategories AS (
  -- Anchor member: start with the categories
  SELECT
    'Short' AS duration_category,
    0 AS min_duration,
    10 AS max_duration

  UNION ALL

  SELECT
    'Medium' AS duration_category,
    10 AS min_duration,
    30 AS max_duration

  UNION ALL

  SELECT
    'Long' AS duration_category,
    30 AS min_duration,
    NULL AS max_duration

),
TripDurations AS (
  SELECT
    CASE
      WHEN duration_minutes < 10 THEN 'Short'
      WHEN duration_minutes >= 10 AND duration_minutes <= 30 THEN 'Medium'
      ELSE 'Long'
    END AS duration_category,
    COUNT(*) AS trip_count
  FROM
    `bigquery-public-data.austin_bikeshare.bikeshare_trips`
  GROUP BY
    duration_category
)

SELECT
  CONCAT('Duration Category: ', dc.duration_category,
         ' - Trips: ', CAST(td.trip_count AS STRING)) AS formatted_result
FROM
  DurationCategories dc
LEFT JOIN
  TripDurations td ON dc.duration_category = td.duration_category
ORDER BY
  dc.min_duration
LIMIT 5;''').to_dataframe()

df.head()

,formatted_result
0,Duration Category: Short - Trips: 996712
1,Duration Category: Medium - Trips: 774202
2,Duration Category: Long - Trips: 500239


**USER DEFINED FUNCTION**

In [34]:
%%bigquery --project data-engineer-next-1

CREATE TEMP FUNCTION calculate_fare(duration_minutes INT64)
RETURNS FLOAT64 AS (
  CASE
    WHEN duration_minutes <= 30 THEN 1.00
    ELSE 1.00 + CEIL((duration_minutes - 30) / 10) * 0.50
  END
);

SELECT
  start_station_name,
  duration_minutes,
  calculate_fare(duration_minutes) AS fare
FROM
  `bigquery-public-data.austin_bikeshare.bikeshare_trips`
ORDER BY fare DESC

LIMIT 5;


Query is running:   0%|          |

Downloading:   0%|          |

,start_station_name,duration_minutes,fare
0,Lakeshore/Pleasant Valley,34238,1711.5
1,Dean Keeton/Speedway,25510,1275.0
2,2nd/Congress,22993,1149.5
3,6th/Chalmers,22964,1148.0
4,Riverside/South Lamar,21748,1087.0
